# Classifying between Imagination and Perception

For each task, orthographic, pictorial or audio, we classify to see whether the subject is imagining or perceiving the stimulus. The task selection should be done in the batch_mill main script. An output csv file is saved with the classification results. In this example, logistic regression is implemented. However, there is the option to implement CSP, PCA and XGBoost which are currently commented out in the classification section.

In [42]:
import mne
import numpy as np
import pandas as pd
from mne.decoding import CSP
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import numpy as np
from scipy import integrate, stats
import antropy as ant

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

In [43]:
subject = '12'
session = '1'
task = 'pictorial' 
iterations = 50 

### Here the task selection is used to set other parameters
The duration is the length of the perception trial. The tag is used to identify epochs based on their event_ids.

In [44]:
if task == 'pictorial':
    tag = 'p'
    duration = 3
elif task == 'orthographic':
    print("orthographic decoding task")
    tag='t'
    duration = 3
elif task == 'audio':
    tag='s'
    duration = 2

### Loading up the data
To make the perception and imagination trials equal, we crop the imagination epochs to be of equal length to the perception duration. This is 2 seconds for audio and 3 seconds for the two visual modalities.

In [45]:
# load up files for one subject for one task of img vs. perc
# imagine_pictorial vs. perception_pictorial



perception_path = 'X:\\CompSci\\ResearchProjects\\EJONeill\\Neuroimaging\\multisensoryeeg\\processed_eeg\\epochs\\perception_'+task+'\\'
imagine_path = 'X:\\CompSci\\ResearchProjects\\EJONeill\\Neuroimaging\\multisensoryeeg\\processed_eeg\\epochs\\imagine_'+task+'\\'
datapoint = subject+'_'+session+ '_epo.fif'

perception_epochs = mne.read_epochs(perception_path + datapoint)
perception_events = mne.read_events(perception_path+ datapoint)
perception_epochs = perception_epochs.crop(tmin=0, tmax=duration)

imagination_epochs = mne.read_epochs(imagine_path + datapoint)
imagination_events = mne.read_events(imagine_path+ datapoint)
imagination_epochs = imagination_epochs.crop(tmin=0, tmax=duration)
epochs = mne.concatenate_epochs([perception_epochs,imagination_epochs])


Reading X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\multisensoryeeg\processed_eeg\epochs\perception_pictorial\12_1_epo.fif ...
    Found the data of interest:
        t =       0.00 ...    3000.00 ms
        0 CTF compensation matrices available
Not setting metadata
109 matching events found
No baseline correction applied
0 projection items activated


C:\Users\hlw69\AppData\Local\Temp\ipykernel_12156\1644791318.py:11: RuntimeWarning: This filename (X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\multisensoryeeg\processed_eeg\epochs\perception_pictorial\12_1_epo.fif) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  perception_events = mne.read_events(perception_path+ datapoint)


Reading X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\multisensoryeeg\processed_eeg\epochs\imagine_pictorial\12_1_epo.fif ...
    Found the data of interest:
        t =       0.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
111 matching events found
No baseline correction applied
0 projection items activated


C:\Users\hlw69\AppData\Local\Temp\ipykernel_12156\1644791318.py:15: RuntimeWarning: This filename (X:\CompSci\ResearchProjects\EJONeill\Neuroimaging\multisensoryeeg\processed_eeg\epochs\imagine_pictorial\12_1_epo.fif) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  imagination_events = mne.read_events(imagine_path+ datapoint)
C:\Users\hlw69\AppData\Local\Temp\ipykernel_12156\1644791318.py:17: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs([perception_epochs,imagination_epochs])


Not setting metadata
220 matching events found
No baseline correction applied


### Dividing Epochs into Perception and Classification
We use the combine_event_ids to merge together all the perception epochs (for the three semantic categories), then do the same again for imagination. These two groups are renamed to be 'perception' and 'imagination'.

In [46]:
# Here we check that the epoch event ids are as expected
print(epochs.event_id)

{'perc_flower_p': 312, 'perc_penguin_p': 313, 'perc_guitar_p': 314, 'imag_flower_p': 303, 'imag_penguin_p': 304, 'imag_guitar_p': 305}


In [47]:

epochs = mne.epochs.combine_event_ids(epochs, old_event_ids = ['perc_penguin_'+tag,'perc_flower_'+tag,'perc_guitar_'+tag],new_event_id = {'perception':1})
epochs = mne.epochs.combine_event_ids(epochs, old_event_ids = ['imag_penguin_'+tag,'imag_flower_'+tag,'imag_guitar_'+tag], new_event_id = {'imagination':0})
print(epochs)


<Epochs |  220 events (all good), 0 - 3 sec, baseline off, ~639.8 MB, data loaded,
 'perception': 109
 'imagination': 111>


### Feature Extraction, statistical features

In [48]:
class StatisticalDecomposition:

    """Creates a folder "SVM features" containing feature arrays for use in classification by the SVM.
These are saved in .npy file format, and can be loaded using np.load("filename.npy") 
The dimensions of the saved arrays are (n_prompts * n_windows * n_features)
The layout of the features in dim 2 is as follows (where d means delta, and dd double-delta):
[mean, absmean, maximum... d_mean, d_absmean, d_maximum... dd_mean, dd_absmean, dd_maximum ... dd_dfa]
To get a list of feature names (in order), use the function get_feats_list()
For convenience, we also save an array of the labels [goose, thought ... ], with dimension (n_prompts) 
in the same folder as the features. For most experiments, the labels will be identical. 
To use entropy/time series features, follow the installation instructions for this package on GitHub:
https://github.com/raphaelvallat/entropy
"""

  #  def __init__(self, _):
        
  #      self.args = _

    # Features used in classification:

    def mean(self, x):
        return np.mean(x)

    def absmean(self, x):
        return np.mean(np.abs(x))

    def maximum(self, x):
        return np.max(x)

    def absmax(self, x):
        return np.max(np.abs(x))

    def minimum(self, x):
        return np.min(x)

    def absmin(self, x):
        return np.min(np.abs(x)) 

    def minplusmax(self, x):
        return np.max(x) + np.min(x)

    def maxminusmin(self, x):
        return np.max(x) - np.min(x)

    def curvelength(self, x):

        cl = 0
        for i in range(x.shape[0]-1):
            cl += abs(x[i]-x[i+1])
        return cl

    def energy(self, x):
        return np.sum(np.multiply(x,x))

    def nonlinear_energy(self, x):
        # NLE(x[n]) = x**2[n] - x[n+1]*x[n-1]
        x_squared = x[1:-1]**2
        subtrahend = x[2:]*x[:-2]
        return np.sum(x_squared-subtrahend)

    #def ehf(x,prev):
        #(based on Basar et. al. 1983)
        # "prev" is array of values from prior context
        # rms = np.sqrt(np.mean(prev**2))
        # return 2*np.sqrt(2)*(max(x)/rms)  

    def spec_entropy(self, x):
        return ant.spectral_entropy(x,fs,method="welch",normalize=True)

    def integral(self, x):
        return integrate.simps(x)

    def stddeviation(self, x):
        return np.std(x)

    def variance(self, x):
        return np.var(x)

    def skew(self, x):
        return stats.skew(x)

    def kurtosis(self, x):
        return stats.kurtosis(x)

    def sum(self, x):
        return np.sum(x)

    # Added ones: some of these are drawn from the antropy library: https://github.com/raphaelvallat/antropy

    def sample_entropy(self, x):
        x = x.astype(np.float64)
        return ant.sample_entropy(x, order=2, metric='chebyshev')  

    def perm_entropy(self, x):
        return ant.perm_entropy(x, order=3, normalize=True)

    def svd_entropy(self, x):
        return ant.svd_entropy(x, order=3, delay=1, normalize=True)

    def app_entropy(self, x):
        return ant.app_entropy(x, order=2, metric='chebyshev')

    def petrosian(self, x):
        return ant.petrosian_fd(x)

    def katz(self, x):
        return ant.katz_fd(x)

    def higuchi(self, x):
        x = x.astype(np.float64)
        return ant.higuchi_fd(x, kmax=10)

    def rootmeansquare(self, x):
        return np.sqrt(np.mean(x**2))

    def dfa(self, x):
        x = x.astype(np.float64)
        return ant.detrended_fluctuation(x)

    def window_data(self, data:np.ndarray):

        # window the data using a stride length of half the window
        # (AKA half the unit stride):

        w_len = int(fs/8) # windows of quarter the sampling frequency
        stride = int(w_len//2) # stride of half the window
      
        return np.lib.stride_tricks.sliding_window_view(data,w_len,axis=1)[:,::stride]

    def compute_feats(self, windowed_data:np.ndarray):
      #  print("Computing feats")
      #  print("windowed_data ", windowed_data.shape)

        # Takes data of shape (channels, windows, time (window_len)),
        # returns an array of shape (channels, windows, num. features)

        funclist = [ # doesn't contain EHF since that must be added later
            self.mean,
            self.absmean,
            self.maximum,
            self.absmax,
            self.minimum,
            self.absmin,
            self.minplusmax,
            self.maxminusmin,
            self.curvelength,
            self.energy,
            self.nonlinear_energy,
            self.integral,
            self.stddeviation,
            self.variance,
            self.skew,
            self.kurtosis,
            self.sum,
            self.spec_entropy,
            self.sample_entropy,
            self.perm_entropy,
            self.svd_entropy,
            self.app_entropy,
            self.petrosian,
            self.katz,
            self.higuchi,
            self.rootmeansquare,
            self.dfa,
            ]

        channels, windows, time = np.shape(windowed_data)
        print(channels, windows, time)
    
        decomposed_data = np.zeros((channels, windows, len(funclist)))
       # print(" Decomposed ", decomposed_data.shape)

        for i in range(len(funclist)):
        #    print(i , len(funclist))
       #     print("windowed data shape ", windowed_data.shape)
            decomposed_data[:,:,i] = np.apply_along_axis(funclist[i], 2, windowed_data)

        return decomposed_data

    def add_deltas(self, feats_array:np.ndarray):

        deltas = np.diff(feats_array,axis=1)
        print(deltas.shape)
        double_deltas = np.diff(deltas,axis=1)
        print(double_deltas.shape)
        feats_array = np.hstack((feats_array[:,2:,:],deltas[:,1:,:],double_deltas))
    #    print("feats_array ", feats_array.shape)
    #    print("reshapred feats ", feats_array.reshape(feats_array.shape[0],-1).shape)
        # resize (channels, windows, num. features) as (channels, windows*num. features):
        return feats_array.reshape(feats_array.shape[0],-1)

    def process_epochs(self, epoch):
        
        epoch = self.window_data(epoch)
       # print("after windowing ", epoch.shape)
        epoch = self.compute_feats(epoch)
       # print("shape of epochs after compute ", epoch.shape)
        epoch = self.add_deltas(epoch)
       # print("shape of epochs after deltas ", epoch.shape)
        
        return epoch


In [51]:
sd = StatisticalDecomposition()

fs = epochs.info['sfreq']

          #  num_feats = num_feats#27*4*3*5 # this needs automating (feats*d+dd*windows)
X = epochs.get_data()
            
    
exemplars, channels, time = epochs.get_data().shape
tup = (exemplars, channels,3645)#,2349) #3645)
print(tup)
decomposed_data = np.empty(tup)
            
for i in tqdm(range(exemplars)):
    print("Exemplars ",  decomposed_data[i].shape)
    decomposed_data[i] = sd.process_epochs(epochs.next())

decomposed_data = decomposed_data.reshape(decomposed_data.shape[0],-1)
#warnings.simplefilter("default")
            
           # labels = np.array([i for i in epochs._metadata['event_name']])

            #if self.args['save']:
             #   np.save(X, decomposed_data)
            #    np.save(y, labels)



(220, 124, 3645)


  0%|                                                                                          | 0/220 [00:00<?, ?it/s]

Exemplars  (124, 3645)
124 47 128


C:\Users\hlw69\Anaconda3\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
  0%|▎                                                                                 | 1/220 [00:10<39:35, 10.85s/it]

(124, 46, 27)
(124, 45, 27)
Exemplars  (124, 3645)
124 47 128


...

C:\Users\hlw69\Anaconda3\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
 99%|███████████████████████████████████████████████████████████████████████████████▎| 218/220 [39:11<00:20, 10.47s/it]

(124, 46, 27)
(124, 45, 27)
Exemplars  (124, 3645)
124 47 128


C:\Users\hlw69\Anaconda3\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 128, using nperseg = 128
  warnings.warn('nperseg = {0:d} is greater than input length '
100%|███████████████████████████████████████████████████████████████████████████████▋| 219/220 [39:22<00:10, 10.79s/it]

(124, 46, 27)
(124, 45, 27)
Exemplars  (124, 3645)


StopIteration: 

In [52]:
decomposed_data = decomposed_data.reshape(decomposed_data.shape[0],-1)
print(decomposed_data.shape)

(220, 451980)


### We create the labels for classification
#### 1 = perception
#### 0 = imagination

In [53]:
labels = epochs.events[:, -1]
print("Labels", labels)
print("Shape of epoch data ", epochs.get_data().shape)



Labels [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Shape of epoch data  (220, 124, 3073)


## Classification
Here, stratified cross validation is implemented with 5 fold. 50 iterations are set. However, the amount of iterations can be edited in the parameter settings.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression


data = decomposed_data
#data = epochs.get_data()
clf = make_pipeline(
  #  model
    Scaler(epochs.info),
  #  psd,
 #   UnsupervisedSpatialFilter(PCA(124), average=False),  # this has to be done due to this error: https://github.com/mne-tools/mne-python/issues/9094
#    csp,
    Vectorizer(),
   # svm.SVC()
  #  LogisticRegression(solver='liblinear')  # liblinear is faster than lbfgs
)

clf = AdaBoostClassifier(n_estimators=100)
#clf =  LogisticRegression(solver='liblinear') 


accuracies = []
from random import randint # ensure that different random_state each time
for i in range(iterations):
    scores = cross_val_score(clf, data, labels, cv=5)
    print(f"Classifier = {scores}. Mean accuracy = {scores.mean()}")
    accuracies.append(scores)

#scores = cross_val_score(clf, data, labels, cv=5)
#print(f"AdaBoost = {scores}. Mean accuracy = {scores.mean()}")

Classifier = [0.75       0.72727273 0.75       0.79545455 0.79545455]. Mean accuracy = 0.7636363636363637
Classifier = [0.75       0.72727273 0.75       0.79545455 0.79545455]. Mean accuracy = 0.7636363636363637
Classifier = [0.75       0.72727273 0.75       0.79545455 0.79545455]. Mean accuracy = 0.7636363636363637
Classifier = [0.75       0.72727273 0.75       0.79545455 0.79545455]. Mean accuracy = 0.7636363636363637
Classifier = [0.75       0.72727273 0.75       0.79545455 0.79545455]. Mean accuracy = 0.7636363636363637
Classifier = [0.75       0.72727273 0.75       0.79545455 0.79545455]. Mean accuracy = 0.7636363636363637
Classifier = [0.75       0.72727273 0.75       0.79545455 0.79545455]. Mean accuracy = 0.7636363636363637


In [13]:
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.pipeline import make_pipeline
from mne.decoding import CSP, Scaler, Vectorizer, cross_val_multiscore,PSDEstimator,UnsupervisedSpatialFilter
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.metrics import plot_confusion_matrix, accuracy_score
import matplotlib.pyplot as plt



data = decomposed_data
#data =  epochs.get_data()
#data = data.reshape(data.shape[0],data.shape[1]*data.shape[2])
#print("Should be 2D if using XGBoost ", data.shape)
#X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

#psd = mne.decoding.PSDEstimator()

csp = CSP(n_components=4, norm_trace=False, reg = 'empirical')
print("psd created")

#model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

clf = make_pipeline(
  #  model
    Scaler(epochs.info),
  #  psd,
 #   UnsupervisedSpatialFilter(PCA(124), average=False),  # this has to be done due to this error: https://github.com/mne-tools/mne-python/issues/9094
#    csp,
    Vectorizer(),
   # svm.SVC()
    LogisticRegression(solver='liblinear')  # liblinear is faster than lbfgs
)

target_names = ['perception', 'imagination']
#linscores = cross_validate(linreg, X_train, y_train, scoring=scoring, return_estimator=True)

# Cross validator
accuracies = []
from random import randint # ensure that different random_state each  time
for i in range(iterations):
    scores = cross_val_score(clf, data, labels, cv=5)
    print(f"Classifier = {scores}. Mean accuracy = {scores.mean()}")
    accuracies.append(scores)
   # print(i)
   # cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=randint(1,50))
   # preds = np.empty(len(labels))
   # for train, test in cv.split(data, labels):
   #     clf.fit(data[train], labels[train])
   #     preds[test] = clf.predict(data[test])
       # report = classification_report(labels[test], preds[test], target_names=target_names)
      #  print("Results for training data : \n ", report)
     #   plot_confusion_matrix(clf, data[test], labels[test])  
      #  plt.show()
    #    acc = accuracy_score(preds[test], labels[test])
     #   print(acc)
      #  accuracies.append(acc)

print("Classification accuracy: %f " % (np.mean(accuracies)))

psd created


ValueError: info had 124 data channels but X has 176 channels

In [ ]:
id = subject+'_'+session
pipe = clf.steps[1:]
std = str(np.std(accuracies))
mean = str(np.mean(accuracies))
print("results for ", id, "using ", pipe, " are mean and std ", mean, std)
with open('perception_vs_imagination_'+task+'.csv','a') as fd:
  #  fd.write(str(np.mean(accuracies)))
    fd.write(f"{id}, {pipe},{mean}, {std}, {iterations}\n")

In [ ]:
#print(clf)

In [ ]:
#target_names = ['perception', 'imagination']
# Classification report
#report = classification_report(y_train, preds, target_names=target_names)
#print("Results for training data : \n ", report)

# test on unseen data
#unseen_preds = np.empty(len(y_test))
#unseen_preds = clf.predict(X_test)

#clsf_report = pd.DataFrame(classification_report(y_test, unseen_preds, target_names=target_names, output_dict=True)).transpose()
#clsf_report.to_csv(subject+'_'+session+'_ClassificationReport.csv', index= True)
#print("Results for unseen test data : \n ", clsf_report)


In [ ]:
#clsf_report

In [ ]:
vviq = [4.25, 3.1, 4.9,  3.3, 3.2, 4.2, 4.1, 3.5, 3.6, 3.1]
bais = [4.8,  4.2, 6.33, 4,   3.8, 5,  5.7, 4,    5.6, 3.8]

from scipy import stats
# check to see whether they violate assumptions of normality
print(stats.shapiro(vviq))
print(stats.shapiro(bais))

#print(stats.ttest_rel(vviq, bais))

from scipy.stats.stats import pearsonr, spearmanr
print(spearmanr(vviq,bais))